In [1]:
import pandas as pd
import requests
import urllib
import json
from urllib.parse import urlencode
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import datetime
from dateutil.parser import parse
import pickle

In [2]:
def dataset_txt_file_download(link, file_name):
  base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
  public_key = link
  final_url = base_url + urlencode(dict(public_key=public_key))
  response = requests.get(final_url)
  download_url = response.json()['href']
  download_response = requests.get(download_url)

  with open(file_name, 'wb') as f:
    f.write(download_response.content)

  df = pd.read_csv(file_name, sep="\t", header = None)
  return df

#Brightkite

Загрузка

In [3]:
brightkite_total_checkin_df = dataset_txt_file_download('https://disk.yandex.ru/d/eZtwKKyB1NI9oQ', 'Brightkite_totalCheckins.txt')

In [4]:
brightkite_total_checkin_df.columns = ['user', 'check-in time', 'latitude', 'longitude', 'location_id']
brightkite_total_checkin_df

,user,check-in time,latitude,longitude,location_id
0,0,2010-10-17T01:48:53Z,39.747652,-104.992510,88c46bf20db295831bd2d1718ad7e6f5
1,0,2010-10-16T06:02:04Z,39.891383,-105.070814,7a0f88982aa015062b95e3b4843f9ca2
2,0,2010-10-16T03:48:54Z,39.891077,-105.068532,dd7cd3d264c2d063832db506fba8bf79
3,0,2010-10-14T18:25:51Z,39.750469,-104.999073,9848afcc62e500a01cf6fbf24b797732f8963683
4,0,2010-10-14T00:21:47Z,39.752713,-104.996337,2ef143e12038c870038df53e0478cefc
...,...,...,...,...,...
4747282,58222,2009-01-23T02:30:34Z,33.833333,35.833333,9f6b83bca22411dd85460384f67fcdb0
4747283,58224,2009-01-03T15:06:54Z,33.833333,35.833333,9f6b83bca22411dd85460384f67fcdb0
4747284,58225,2009-01-20T13:58:14Z,33.833333,35.833333,9f6b83bca22411dd85460384f67fcdb0
4747285,58226,2009-01-20T13:30:09Z,33.833333,35.833333,9f6b83bca22411dd85460384f67fcdb0


Удаление Nan

In [5]:
brightkite_total_checkin_df = brightkite_total_checkin_df.dropna()

In [6]:
brightkite_total_checkin_df.longitude.max()

179.999

Группировка координат по областям

In [7]:
brightkite_total_checkin_df.loc[:, 'longitude_bin'] = pd.qcut(brightkite_total_checkin_df.longitude, q=15, labels = [i for i in range(0, 15)])

<ipython-input-7-c606ba8373fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brightkite_total_checkin_df.loc[:, 'longitude_bin'] = pd.qcut(brightkite_total_checkin_df.longitude, q=15, labels = [i for i in range(0, 15)])


In [8]:
brightkite_total_checkin_df.loc[:, 'latitude_bin'] = pd.qcut(brightkite_total_checkin_df.latitude, q=15, labels = [i for i in range(0, 15)])

<ipython-input-8-c4da673f24d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brightkite_total_checkin_df.loc[:, 'latitude_bin'] = pd.qcut(brightkite_total_checkin_df.latitude, q=15, labels = [i for i in range(0, 15)])


In [9]:
brightkite_total_checkin_df.loc[:, 'longitude_bin'] = brightkite_total_checkin_df['longitude_bin'].astype('int64')

In [10]:
brightkite_total_checkin_df.loc[:, 'latitude_bin'] = brightkite_total_checkin_df['latitude_bin'].astype('int64')

In [11]:
len(brightkite_total_checkin_df['longitude_bin'].unique())

15

In [12]:
len(brightkite_total_checkin_df['latitude_bin'].unique())

15

In [13]:
brightkite_total_checkin_df.dtypes

user                int64
check-in time      object
latitude          float64
longitude         float64
location_id        object
longitude_bin    category
latitude_bin     category
dtype: object

In [14]:
brightkite_total_checkin_df.loc[:, 'location_id_bin'] = list(zip(brightkite_total_checkin_df.latitude_bin, brightkite_total_checkin_df.longitude_bin))

<ipython-input-14-5b609133ceb0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brightkite_total_checkin_df.loc[:, 'location_id_bin'] = list(zip(brightkite_total_checkin_df.latitude_bin, brightkite_total_checkin_df.longitude_bin))


In [15]:
brightkite_total_checkin_df.loc[:, 'place'] = list(zip(brightkite_total_checkin_df.latitude, brightkite_total_checkin_df.longitude))

<ipython-input-15-fd416bf27b13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brightkite_total_checkin_df.loc[:, 'place'] = list(zip(brightkite_total_checkin_df.latitude, brightkite_total_checkin_df.longitude))


In [16]:
brightkite_total_checkin_df.loc[:, 'location_id_bin'] = brightkite_total_checkin_df['location_id_bin'].astype('str')

In [17]:
brightkite_total_checkin_df

,user,check-in time,latitude,longitude,location_id,longitude_bin,latitude_bin,location_id_bin,place
0,0,2010-10-17T01:48:53Z,39.747652,-104.992510,88c46bf20db295831bd2d1718ad7e6f5,3,9,"(9, 3)","(39.747652, -104.99251)"
1,0,2010-10-16T06:02:04Z,39.891383,-105.070814,7a0f88982aa015062b95e3b4843f9ca2,3,9,"(9, 3)","(39.891383, -105.070814)"
2,0,2010-10-16T03:48:54Z,39.891077,-105.068532,dd7cd3d264c2d063832db506fba8bf79,3,9,"(9, 3)","(39.891077, -105.068532)"
3,0,2010-10-14T18:25:51Z,39.750469,-104.999073,9848afcc62e500a01cf6fbf24b797732f8963683,3,9,"(9, 3)","(39.750469, -104.999073)"
4,0,2010-10-14T00:21:47Z,39.752713,-104.996337,2ef143e12038c870038df53e0478cefc,3,9,"(9, 3)","(39.752713, -104.996337)"
...,...,...,...,...,...,...,...,...,...
4747282,58222,2009-01-23T02:30:34Z,33.833333,35.833333,9f6b83bca22411dd85460384f67fcdb0,12,4,"(4, 12)","(33.833333, 35.833333)"
4747283,58224,2009-01-03T15:06:54Z,33.833333,35.833333,9f6b83bca22411dd85460384f67fcdb0,12,4,"(4, 12)","(33.833333, 35.833333)"
4747284,58225,2009-01-20T13:58:14Z,33.833333,35.833333,9f6b83bca22411dd85460384f67fcdb0,12,4,"(4, 12)","(33.833333, 35.833333)"
4747285,58226,2009-01-20T13:30:09Z,33.833333,35.833333,9f6b83bca22411dd85460384f67fcdb0,12,4,"(4, 12)","(33.833333, 35.833333)"


In [18]:
len(brightkite_total_checkin_df.location_id_bin.unique())

207

In [19]:
unique_locations = pd.DataFrame()

In [20]:
brightkite_total_checkin_df = brightkite_total_checkin_df.dropna()

In [21]:
len(brightkite_total_checkin_df.location_id_bin.unique())

207

In [22]:
brightkite_total_checkin_df.location_id_bin.value_counts()

location_id_bin
(0, 10)     256193
(9, 3)      148747
(14, 12)    147761
(3, 2)      145890
(4, 1)      127624
             ...  
(14, 5)          3
(0, 5)           3
(2, 0)           2
(13, 8)          2
(14, 7)          1
Name: count, Length: 207, dtype: int64

Загрузка информации о графе

In [23]:
brightkite_edges_df = dataset_txt_file_download('https://disk.yandex.ru/i/LOYwqXxn13LBug', 'Brightkite_edges.txt')

In [24]:
brightkite_edges_df.columns = ['1st friend', '2nd friend']
brightkite_edges_df

,1st friend,2nd friend
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
428151,58225,58226
428152,58225,58227
428153,58226,58220
428154,58226,58225


Отфильтровываем пользователей, которых нет в датасете с событиями

In [25]:
brightkite_edges_df = brightkite_edges_df[brightkite_edges_df['1st friend'].isin(brightkite_total_checkin_df.user.unique())].reset_index().drop(['index'], axis=1)

In [26]:
brightkite_edges_df = brightkite_edges_df[brightkite_edges_df['2nd friend'].isin(brightkite_total_checkin_df.user.unique())].reset_index().drop(['index'], axis=1)

In [27]:
le = LabelEncoder()
brightkite_total_checkin_df.location_id_bin = le.fit_transform(brightkite_total_checkin_df.location_id_bin.values)

Вывод областей и всех координат, которые в них попали

In [28]:
brightkite_sectors = brightkite_total_checkin_df.groupby('location_id_bin').agg({'place':lambda x: list(x)}).reset_index()

In [29]:
brightkite_sectors

,location_id_bin,place
0,0,"[(-18.060833, -163.193333), (-18.060833, -163...."
1,1,"[(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0..."
2,2,"[(-0.000131, 0.007579), (-0.000131, 0.007579),..."
3,3,"[(-30.0, 26.0), (-33.966667, 22.45), (-25.8554..."
4,4,"[(-8.512955, 115.26106), (-8.514138, 115.26392..."
...,...,...
202,202,"[(39.799443, -89.648965), (39.73221, -86.2707)..."
203,203,"[(40.0, -82.883333), (39.961176, -82.998794), ..."
204,204,"[(40.491389, -80.232778), (40.495844, -80.2531..."
205,205,"[(39.877328, -75.240086), (39.877328, -75.2400..."


Сортировка событий по времени наступления

In [30]:
brightkite_total_checkin_df = brightkite_total_checkin_df.sort_values('check-in time')

In [31]:
brightkite_total_checkin_df['check-in time'] = pd.to_datetime(brightkite_total_checkin_df['check-in time'])

In [32]:
brightkite_total_checkin_df["check-in time"] = brightkite_total_checkin_df['check-in time'].map(lambda x: int(x.timestamp()))

Группировка по пользователям, получение последовательностей для каждого пользователя

In [33]:
brightkite_users = brightkite_total_checkin_df.groupby('user').agg({'location_id_bin':lambda x: list(x), 'check-in time':lambda x: list(x)}).reset_index()

In [34]:
brightkite_users = brightkite_users[brightkite_users.user.isin(brightkite_edges_df['1st friend'].unique())].reset_index()

Получение всех уникальных событий, которые встречаются в последовательностях

In [35]:
common_actions_set = []
for locations in brightkite_users.location_id_bin:
  common_actions_set.extend(set(locations))
common_actions_set = set(common_actions_set)

In [36]:
brightkite_item_id = {}
index = 0
for location in common_actions_set:
  brightkite_item_id[location] = index
  index += 1

In [37]:
brightkite_total_checkin_df = brightkite_total_checkin_df[brightkite_total_checkin_df.user.isin(brightkite_edges_df['1st friend'].unique())].reset_index()

Отфильтровываем последовательности, длина которых меньше 3

In [38]:
brightkite_users = brightkite_users[brightkite_users.location_id_bin.map(lambda x: len(x) > 3)]

In [39]:
brightkite_edges_df = brightkite_edges_df[brightkite_edges_df['1st friend'].isin(brightkite_users.user.unique())].reset_index().drop(['index'], axis=1)

In [40]:
brightkite_edges_df = brightkite_edges_df[brightkite_edges_df['2nd friend'].isin(brightkite_users.user.unique())].reset_index().drop(['index'], axis=1)

Перезаписываем id юзеров по порядку

In [41]:
brightkite_user_id = {}
index = 0
for user in brightkite_users.user:
  brightkite_user_id[user] = index
  index += 1

In [42]:
brightkite_users = brightkite_users.reset_index()

Перезаписываем id юзеров в датасете с графом

In [43]:
for edge in range(0, brightkite_edges_df.shape[0]):
  brightkite_edges_df.loc[edge, '1st friend'] = brightkite_user_id[brightkite_edges_df.loc[edge, '1st friend']]

In [44]:
for edge in range(0, brightkite_edges_df.shape[0]):
  brightkite_edges_df.loc[edge, '2nd friend'] = brightkite_user_id[brightkite_edges_df.loc[edge, '2nd friend']]

In [45]:
for user in range(0, brightkite_users.shape[0]):
  brightkite_users.loc[user, 'user'] = brightkite_user_id[brightkite_users.loc[user, 'user']]

Отбираем 20000 юзеров

In [46]:
short_brightkite_user_activity = brightkite_users[brightkite_users['user'] <= 20000]

In [47]:
with open('main_brightkite_all_ids.pkl', 'wb') as file:
    pickle.dump([brightkite_user_id, brightkite_item_id], file)

In [48]:
short_brightkite_user_activity = short_brightkite_user_activity.drop(['index'], axis=1)

In [49]:
short_brightkite_user_activity = short_brightkite_user_activity.drop(['level_0'], axis=1)

In [50]:
brightkite_total_checkin_df = brightkite_total_checkin_df.drop(['index'], axis=1)

Отбираем ребра графа, которым принадлежат отобранные пользователи

In [51]:
short_brightkite_edges = brightkite_edges_df[(brightkite_edges_df['1st friend'] <= 20000 )& (brightkite_edges_df['2nd friend'] <= 20000)]

In [52]:
short_brightkite_edges[short_brightkite_edges['2nd friend'] == 20000]

,1st friend,2nd friend


In [53]:
short_brightkite_user_activity.shape[0]

20001

Сохраняем информацию о последовательностях и о ребрах социального графа

In [54]:
short_brightkite_user_activity.to_csv('brightkite_user_activity.csv')
short_brightkite_edges.to_csv('brightkite_edges.csv')

In [55]:
short_brightkite_user_activity_target = pd.DataFrame()
short_brightkite_user_activity_target['user'] = short_brightkite_user_activity['user']
short_brightkite_user_activity_target['target_test'] = short_brightkite_user_activity['location_id_bin'].map(lambda x : x[-1])
short_brightkite_user_activity_target['target_train'] = short_brightkite_user_activity['location_id_bin'].map(lambda x : x[-2])

In [56]:
short_brightkite_user_activity_target = short_brightkite_user_activity_target.reset_index()

In [57]:
short_brightkite_user_activity_target.to_csv('brightkite_user_activity_target.csv')

In [58]:
import json
short_brightkite_user_activity = pd.read_csv('brightkite_user_activity.csv')

In [59]:
short_brightkite_user_activity.shape[0]

20001

In [60]:
brightkite_edges_df = pd.read_csv('brightkite_edges.csv')

Код сохранения последовательностей пользователей в текстовый файл

In [ ]:
file = open('brightkite_user_activity.txt', 'w')
for i in range(0, 20001):
  item_time_string = ""
  locations_list = json.loads(short_brightkite_user_activity.loc[i, 'location_id_bin'])
  time_list = json.loads(short_brightkite_user_activity.loc[i, 'check-in time'])
  for j in range(len(locations_list)):
    location = locations_list[j]
    time = time_list[j]
    item_time_string += f'{location}:{time}'
    if j < len(short_brightkite_user_activity.loc[i, 'location_id_bin']) - 1:
      item_time_string += " "
  user = short_brightkite_user_activity.loc[i, 'user']
  user_string = f'{user}: '
  user_string += item_time_string
  if i < 20000:
    user_string += "\n"
  file.write(user_string)

# Gowalla

In [61]:
import torch

In [62]:
gowalla_total_checkin_df = dataset_txt_file_download('https://disk.yandex.ru/d/KVm7gIom6YooWA', 'Gowalla_totalCheckins.txt')

In [63]:
gowalla_total_checkin_df.columns = ['user', 'check-in time', 'latitude', 'longitude', 'location_id']
gowalla_total_checkin_df

,user,check-in time,latitude,longitude,location_id
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878
...,...,...,...,...,...
6442887,196578,2010-06-11T13:32:26Z,51.742988,-0.488065,906885
6442888,196578,2010-06-11T13:26:45Z,51.746492,-0.490780,965121
6442889,196578,2010-06-11T13:26:34Z,51.741916,-0.496729,1174322
6442890,196585,2010-10-08T21:01:49Z,50.105516,8.571525,471724


In [64]:
gowalla_total_checkin_df['longitude_bin'] = pd.qcut(gowalla_total_checkin_df.longitude, q=15, labels = [i for i in range(0, 15)])

In [65]:
gowalla_total_checkin_df['latitude_bin'] = pd.qcut(gowalla_total_checkin_df.latitude, q=15, labels = [i for i in range(0, 15)])

In [66]:
gowalla_total_checkin_df['longitude_bin'] = gowalla_total_checkin_df['longitude_bin'].astype('int64')

In [67]:
gowalla_total_checkin_df['latitude_bin'] = gowalla_total_checkin_df['latitude_bin'].astype('int64')

In [68]:
len(gowalla_total_checkin_df['longitude_bin'].unique())

15

In [69]:
len(gowalla_total_checkin_df['latitude_bin'].unique())

15

In [70]:
gowalla_total_checkin_df.dtypes

user               int64
check-in time     object
latitude         float64
longitude        float64
location_id        int64
longitude_bin      int64
latitude_bin       int64
dtype: object

In [71]:
gowalla_total_checkin_df['location_id_bin'] = list(zip(gowalla_total_checkin_df.latitude_bin, gowalla_total_checkin_df.longitude_bin))

In [72]:
gowalla_total_checkin_df['place'] = list(zip(gowalla_total_checkin_df.latitude, gowalla_total_checkin_df.longitude))

In [73]:
gowalla_total_checkin_df['location_id_bin'] = gowalla_total_checkin_df['location_id_bin'].astype('str')

In [74]:
gowalla_total_checkin_df

,user,check-in time,latitude,longitude,location_id,longitude_bin,latitude_bin,location_id_bin,place
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847,3,1,"(1, 3)","(30.2359091167, -97.7951395833)"
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315,3,2,"(2, 3)","(30.2691029532, -97.7493953705)"
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637,3,1,"(1, 3)","(30.2557309927, -97.7633857727)"
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516,3,2,"(2, 3)","(30.2634181234, -97.7575966669)"
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878,3,2,"(2, 3)","(30.2742918584, -97.7405226231)"
...,...,...,...,...,...,...,...,...,...
6442887,196578,2010-06-11T13:32:26Z,51.742988,-0.488065,906885,9,11,"(11, 9)","(51.7429879141, -0.4880654812)"
6442888,196578,2010-06-11T13:26:45Z,51.746492,-0.490780,965121,9,11,"(11, 9)","(51.7464920601, -0.4907798767)"
6442889,196578,2010-06-11T13:26:34Z,51.741916,-0.496729,1174322,9,11,"(11, 9)","(51.7419158333, -0.49672855)"
6442890,196585,2010-10-08T21:01:49Z,50.105516,8.571525,471724,10,10,"(10, 10)","(50.10551553, 8.57152539)"


In [75]:
len(gowalla_total_checkin_df.location_id_bin.unique())

186

In [76]:
unique_locations = pd.DataFrame()

In [77]:
gowalla_total_checkin_df = gowalla_total_checkin_df.dropna()

In [78]:
len(gowalla_total_checkin_df.location_id_bin.unique())

186

In [79]:
gowalla_total_checkin_df.location_id_bin.value_counts()

location_id_bin
(0, 14)     300878
(2, 3)      284728
(3, 4)      249978
(11, 9)     247301
(13, 13)    220097
             ...  
(3, 11)          1
(4, 10)          1
(11, 0)          1
(14, 5)          1
(13, 5)          1
Name: count, Length: 186, dtype: int64

In [80]:
gowalla_edges_df = dataset_txt_file_download('https://disk.yandex.ru/d/Sr3FpAa7WG4GdA', 'Gowalla_edges.txt')

In [81]:
gowalla_edges_df.columns = ['1st friend', '2nd friend']
gowalla_edges_df

,1st friend,2nd friend
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
1900649,196586,196539
1900650,196587,196540
1900651,196588,196540
1900652,196589,196547


In [82]:
len(gowalla_edges_df['1st friend'].unique())

196591

In [83]:
len(gowalla_edges_df['2nd friend'].unique())

196591

In [84]:
gowalla_edges_df = gowalla_edges_df[gowalla_edges_df['1st friend'].isin(gowalla_total_checkin_df.user.unique())].reset_index().drop(['index'], axis=1)

In [85]:
gowalla_edges_df = gowalla_edges_df[gowalla_edges_df['2nd friend'].isin(gowalla_total_checkin_df.user.unique())].reset_index().drop(['index'], axis=1)

In [86]:
len(gowalla_edges_df['1st friend'].unique())

99563

In [87]:
len(gowalla_edges_df['2nd friend'].unique())

99563

In [88]:
le = LabelEncoder()
gowalla_total_checkin_df.location_id_bin = le.fit_transform(gowalla_total_checkin_df.location_id_bin.values)

In [89]:
gowalla_total_checkin_df[gowalla_total_checkin_df.location_id_bin == 0]

,user,check-in time,latitude,longitude,location_id,longitude_bin,latitude_bin,location_id_bin,place
2381,4,2010-06-26T21:24:44Z,21.333193,-157.920570,28457,0,0,0,"(21.3331932881, -157.9205703735)"
2382,4,2010-06-26T18:46:39Z,21.977912,-159.349480,34954,0,0,0,"(21.9779116414, -159.3494796753)"
2383,4,2010-06-26T04:29:23Z,22.223604,-159.467304,1320020,0,0,0,"(22.223603717, -159.46730375)"
2384,4,2010-06-24T22:38:09Z,22.212546,-159.498158,369919,0,0,0,"(22.2125465, -159.4981578167)"
2385,4,2010-06-24T22:32:56Z,22.213026,-159.496358,603934,0,0,0,"(22.2130263667, -159.4963583167)"
...,...,...,...,...,...,...,...,...,...
6430534,194396,2010-09-07T03:37:51Z,21.298361,-158.072877,507267,0,0,0,"(21.298361101, -158.0728769302)"
6430535,194396,2010-09-05T11:37:43Z,21.279179,-157.787873,75860,0,0,0,"(21.27917885, -157.7878725167)"
6430536,194396,2010-09-03T05:27:34Z,21.332288,-157.874075,86095,0,0,0,"(21.3322883, -157.8740748)"
6430537,194396,2010-09-02T06:19:27Z,21.288157,-157.827328,2904843,0,0,0,"(21.28815682, -157.82732791)"


In [90]:
gowalla_sectors = gowalla_total_checkin_df.groupby('location_id_bin').agg({'place':lambda x: list(x)}).reset_index()

In [91]:
gowalla_sectors

,location_id_bin,place
0,0,"[(21.3331932881, -157.9205703735), (21.9779116..."
1,1,"[(0.3381589, 6.73802894), (0.3292855, 6.725065..."
2,2,"[(-32.809649533, 17.890348483), (-32.807609917..."
3,3,"[(-34.119453217, 18.390906633), (-34.10239541,..."
4,4,"[(-33.9466514069, 151.1671169551), (-33.936974..."
...,...,...
181,181,"[(42.947941619, -87.9032850266), (44.861131567..."
182,182,"[(42.9731706167, -83.7379660667), (42.65519243..."
183,183,"[(43.6686418032, -79.3942277878), (43.66801906..."
184,184,"[(42.6833348833, -73.83982765), (42.6835050833..."


In [92]:
gowalla_total_checkin_df.location_id_bin.unique().tolist().index(0)

20

In [93]:
gowalla_total_checkin_df.location_id.unique()

array([  22847,  420315,  316637, ...,  845387, 1341442, 4555073])

In [94]:
gowalla_total_checkin_df = gowalla_total_checkin_df.sort_values('check-in time')

In [95]:
gowalla_total_checkin_df['check-in time'] = pd.to_datetime(gowalla_total_checkin_df['check-in time'])

In [96]:
gowalla_total_checkin_df["check-in time"] = gowalla_total_checkin_df['check-in time'].map(lambda x: int(x.timestamp()))

In [97]:
gowalla_users = gowalla_total_checkin_df.groupby('user').agg({'location_id_bin':lambda x: list(x), 'check-in time':lambda x: list(x)}).reset_index()

In [98]:
gowalla_users = gowalla_users[gowalla_users.user.isin(gowalla_edges_df['1st friend'].unique())].reset_index()

In [99]:
common_actions_set = []
for locations in gowalla_users.location_id_bin:
  common_actions_set.extend(set(locations))
common_actions_set = set(common_actions_set)

In [100]:
gowalla_item_id = {}
index = 0
for location in common_actions_set:
  gowalla_item_id[location] = index
  index += 1

In [101]:
gowalla_total_checkin_df = gowalla_total_checkin_df[gowalla_total_checkin_df.user.isin(gowalla_edges_df['1st friend'].unique())].reset_index()

In [102]:
gowalla_users = gowalla_users[gowalla_users.location_id_bin.map(lambda x: len(x) > 3)]

In [103]:
gowalla_edges_df[gowalla_edges_df['1st friend'] == 0]

,1st friend,2nd friend
0,0,1
1,0,2
2,0,4
3,0,5
4,0,7
...,...,...
555,0,610
556,0,611
557,0,612
558,0,614


In [104]:
gowalla_edges_df = gowalla_edges_df[gowalla_edges_df['1st friend'].isin(gowalla_users.user.unique())].reset_index().drop(['index'], axis=1)

In [105]:
gowalla_edges_df = gowalla_edges_df[gowalla_edges_df['2nd friend'].isin(gowalla_users.user.unique())].reset_index().drop(['index'], axis=1)

In [106]:
gowalla_user_id = {}
index = 0
for user in gowalla_users.user:
  gowalla_user_id[user] = index
  index += 1

In [107]:
gowalla_users = gowalla_users.reset_index()

In [108]:
for edge in range(0, gowalla_edges_df.shape[0]):
  gowalla_edges_df.loc[edge, '1st friend'] = gowalla_user_id[gowalla_edges_df.loc[edge, '1st friend']]

In [109]:
for edge in range(0, gowalla_edges_df.shape[0]):
  gowalla_edges_df.loc[edge, '2nd friend'] = gowalla_user_id[gowalla_edges_df.loc[edge, '2nd friend']]

In [110]:
for user in range(0, gowalla_users.shape[0]):
  gowalla_users.loc[user, 'user'] = gowalla_user_id[gowalla_users.loc[user, 'user']]

In [111]:
short_gowalla_user_activity = gowalla_users[gowalla_users['user'] <= 20000]

In [112]:
short_gowalla_user_activity

,level_0,index,user,location_id_bin,check-in time
0,0,0,0,"[16, 150, 151, 151, 151, 151, 151, 80, 80, 127...","[1274496544, 1274550655, 1274555592, 127463345..."
1,1,1,1,"[24, 35, 35, 35, 173, 173, 173, 173, 173, 185,...","[1278303454, 1278413201, 1278489200, 127856842..."
2,2,2,2,"[99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 9...","[1282754652, 1282754686, 1282754731, 128275479..."
3,3,3,3,"[127, 127, 127, 127, 127, 127, 127, 127, 127, ...","[1272603136, 1272650278, 1272726214, 127291637..."
4,4,4,4,"[80, 80, 80, 80, 80, 80, 16, 80, 80, 80, 80, 8...","[1266430059, 1266548591, 1266628558, 126664391..."
...,...,...,...,...,...
19996,21916,22235,19996,"[183, 183, 183, 183, 183, 183, 183, 183, 183, ...","[1270060878, 1270146339, 1270152785, 127022295..."
19997,21917,22236,19997,"[122, 108, 123, 123, 123, 83, 19, 19, 122, 122...","[1277055954, 1277142904, 1277165789, 127716792..."
19998,21918,22237,19998,"[182, 167, 182, 182, 166, 166, 166, 166, 166, ...","[1277786926, 1277853904, 1278019319, 127809364..."
19999,21919,22238,19999,"[107, 107, 107, 107, 107, 107, 107, 107, 107, ...","[1276367847, 1276376136, 1276377568, 127638858..."


In [113]:
with open('short_gowalla_item_ids.pkl', 'wb') as file:
    pickle.dump(gowalla_item_id, file)

In [114]:
print(len(gowalla_item_id), len(gowalla_user_id))

186 84498


In [115]:
with open('main_gowalla_all_ids.pkl', 'wb') as file:
    pickle.dump([gowalla_user_id, gowalla_item_id], file)

In [116]:
short_gowalla_user_activity = short_gowalla_user_activity.drop(['index'], axis=1)

In [117]:
short_gowalla_user_activity = short_gowalla_user_activity.drop(['level_0'], axis=1)

In [118]:
short_gowalla_user_activity

,user,location_id_bin,check-in time
0,0,"[16, 150, 151, 151, 151, 151, 151, 80, 80, 127...","[1274496544, 1274550655, 1274555592, 127463345..."
1,1,"[24, 35, 35, 35, 173, 173, 173, 173, 173, 185,...","[1278303454, 1278413201, 1278489200, 127856842..."
2,2,"[99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 9...","[1282754652, 1282754686, 1282754731, 128275479..."
3,3,"[127, 127, 127, 127, 127, 127, 127, 127, 127, ...","[1272603136, 1272650278, 1272726214, 127291637..."
4,4,"[80, 80, 80, 80, 80, 80, 16, 80, 80, 80, 80, 8...","[1266430059, 1266548591, 1266628558, 126664391..."
...,...,...,...
19996,19996,"[183, 183, 183, 183, 183, 183, 183, 183, 183, ...","[1270060878, 1270146339, 1270152785, 127022295..."
19997,19997,"[122, 108, 123, 123, 123, 83, 19, 19, 122, 122...","[1277055954, 1277142904, 1277165789, 127716792..."
19998,19998,"[182, 167, 182, 182, 166, 166, 166, 166, 166, ...","[1277786926, 1277853904, 1278019319, 127809364..."
19999,19999,"[107, 107, 107, 107, 107, 107, 107, 107, 107, ...","[1276367847, 1276376136, 1276377568, 127638858..."


In [119]:
short_gowalla_edges = gowalla_edges_df[(gowalla_edges_df['1st friend'] <= 20000 )& (gowalla_edges_df['2nd friend'] <= 20000)]

In [122]:
short_gowalla_user_activity.to_csv('gowalla_user_activity.csv')
short_gowalla_edges.to_csv('gowalla_edges.csv')

In [123]:
short_gowalla_user_activity_target = pd.DataFrame()
short_gowalla_user_activity_target['user'] = short_gowalla_user_activity['user']
short_gowalla_user_activity_target['target_test'] = short_gowalla_user_activity['location_id_bin'].map(lambda x : x[-1])
short_gowalla_user_activity_target['target_train'] = short_gowalla_user_activity['location_id_bin'].map(lambda x : x[-2])

In [124]:
short_gowalla_user_activity_target = short_gowalla_user_activity_target.reset_index()

In [125]:
short_gowalla_user_activity_target.to_csv('user_activity_target.csv')

###File for NACTSeer

In [ ]:
import pandas as pd
short_gowalla_user_activity = pd.read_csv('gowalla_user_activity.csv')
import json

In [ ]:
short_gowalla_user_activity

,Unnamed: 0,user,location_id_bin,check-in time
0,0,0,"[16, 150, 151, 151, 151, 151, 151, 80, 80, 127...","[1274496544, 1274550655, 1274555592, 127463345..."
1,1,1,"[24, 35, 35, 35, 173, 173, 173, 173, 173, 185,...","[1278303454, 1278413201, 1278489200, 127856842..."
2,2,2,"[99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 9...","[1282754652, 1282754686, 1282754731, 128275479..."
3,3,3,"[127, 127, 127, 127, 127, 127, 127, 127, 127, ...","[1272603136, 1272650278, 1272726214, 127291637..."
4,4,4,"[80, 80, 80, 80, 80, 80, 16, 80, 80, 80, 80, 8...","[1266430059, 1266548591, 1266628558, 126664391..."
...,...,...,...,...
19996,19996,19996,"[183, 183, 183, 183, 183, 183, 183, 183, 183, ...","[1270060878, 1270146339, 1270152785, 127022295..."
19997,19997,19997,"[122, 108, 123, 123, 123, 83, 19, 19, 122, 122...","[1277055954, 1277142904, 1277165789, 127716792..."
19998,19998,19998,"[182, 167, 182, 182, 166, 166, 166, 166, 166, ...","[1277786926, 1277853904, 1278019319, 127809364..."
19999,19999,19999,"[107, 107, 107, 107, 107, 107, 107, 107, 107, ...","[1276367847, 1276376136, 1276377568, 127638858..."


In [ ]:
common_actions_set = []
for locations in short_gowalla_user_activity.location_id_bin:
    locations = json.loads(locations)
    common_actions_set.extend(set(locations))

In [ ]:
print(len(set(common_actions_set)))

178


In [ ]:
gowalla_item_id = {}
index = 0
for location in common_actions_set:
  gowalla_item_id[location] = index
  index += 1

In [ ]:
gowalla_user_id = {}
index = 0
for user in short_gowalla_user_activity.user:
  gowalla_user_id[user] = index
  index += 1

In [ ]:
gowalla_edges_df = dataset_txt_file_download('https://disk.yandex.ru/d/Sr3FpAa7WG4GdA', 'Gowalla_edges.txt')

In [ ]:
gowalla_edges_df.columns = ['1st friend', '2nd friend']
gowalla_edges_df

,1st friend,2nd friend
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
1900649,196586,196539
1900650,196587,196540
1900651,196588,196540
1900652,196589,196547


In [ ]:
gowalla_edges_df = gowalla_edges_df[gowalla_edges_df['1st friend'].isin(short_gowalla_user_activity.user.unique())].reset_index().drop(['index'], axis=1)

In [ ]:
gowalla_edges_df = gowalla_edges_df[gowalla_edges_df['2nd friend'].isin(short_gowalla_user_activity.user.unique())].reset_index().drop(['index'], axis=1)

In [ ]:
for user in range(0, short_gowalla_user_activity.shape[0]):
  short_gowalla_user_activity.loc[user, 'user'] = gowalla_user_id[short_gowalla_user_activity.loc[user, 'user']]

In [ ]:
file = open('gowalla_user_activity.txt', 'w')
for i in range(0, 20001):
  item_time_string = ""
  locations_list = json.loads(short_gowalla_user_activity.loc[i, 'location_id_bin'])
  time_list = json.loads(short_gowalla_user_activity.loc[i, 'check-in time'])
  for j in range(len(locations_list)):
    location = locations_list[j]
    time = time_list[j]
    item_time_string += f'{location}:{time}'
    if j < len(short_gowalla_user_activity.loc[i, 'location_id_bin']) - 1:
      item_time_string += " "
  user = short_gowalla_user_activity.loc[i, 'user']
  user_string = f'{user}: '
  user_string += item_time_string
  if i < 20000:
    user_string += "\n"
  file.write(user_string)

In [ ]:
for edge in range(0, gowalla_edges_df.shape[0]):
  gowalla_edges_df.loc[edge, '1st friend'] = gowalla_user_id[gowalla_edges_df.loc[edge, '1st friend']]

In [ ]:
for edge in range(0, gowalla_edges_df.shape[0]):
  gowalla_edges_df.loc[edge, '2nd friend'] = gowalla_user_id[gowalla_edges_df.loc[edge, '2nd friend']]

In [ ]:
graph_file = open('gowalla_graph.txt', 'w')
for i in range(gowalla_edges_df.shape[0]):
  first = gowalla_edges_df.loc[i, '1st friend']
  second = gowalla_edges_df.loc[i, '2nd friend']
  edge_string = f'{first} {second}'
  if i < gowalla_edges_df.shape[0] - 1:
    edge_string += "\n"
  graph_file.write(edge_string)

In [ ]:
with open('main_gowalla_all_ids.pkl', 'wb') as file:
    pickle.dump([gowalla_user_id, gowalla_item_id], file)